# Extração de caracteristicas - studentVle

Esse notebook tem como objetivo realizar extração de caracteristicas da tabela *studentVle* que é uma das mais importantes para entender o engajamento do aluno dentro do curso.

In [4]:
!pip install -qqq pandas_profiling

import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from missingno import matrix

mpl.rcParams['figure.dpi'] = 200
sns.set_style("darkgrid")

In [5]:
arquivos = glob.glob("../../data/raw/*.csv")
arquivos

['../../data/raw/studentAssessment.csv',
 '../../data/raw/studentInfo.csv',
 '../../data/raw/studentVle.csv',
 '../../data/raw/courses.csv',
 '../../data/raw/vle.csv',
 '../../data/raw/studentRegistration.csv',
 '../../data/raw/assessments.csv']

In [7]:
data = {
    a.split('/')[-1][:-4]: pd.read_csv(a)
    for a in arquivos
}
data.keys()

dict_keys(['studentAssessment', 'studentInfo', 'studentVle', 'courses', 'vle', 'studentRegistration', 'assessments'])

## Modelo dos dados

![](../../images/model.png)

### Tabela *studentVle*

In [4]:
data['studentVle']

,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1
...,...,...,...,...,...,...
10655275,GGG,2014J,675811,896943,269,3
10655276,GGG,2014J,675578,896943,269,1
10655277,GGG,2014J,654064,896943,269,3
10655278,GGG,2014J,654064,896939,269,1


In [13]:
studentVle = data['studentVle'].set_index("id_site").join(
    data['vle'].drop(columns=['code_module', 'code_presentation']).set_index("id_site"), 
)
studentVle

,code_module,code_presentation,id_student,date,sum_click,activity_type,week_from,week_to
id_site,,,,,,,,
526721,FFF,2013B,247666,-18,11,homepage,NaN,NaN
526721,FFF,2013B,244114,-18,5,homepage,NaN,NaN
526721,FFF,2013B,243657,-18,16,homepage,NaN,NaN
526721,FFF,2013B,243600,-18,31,homepage,NaN,NaN
526721,FFF,2013B,244816,-18,11,homepage,NaN,NaN
...,...,...,...,...,...,...,...,...
1046866,CCC,2014J,587391,258,1,resource,NaN,NaN
1046866,CCC,2014J,2598961,264,1,resource,NaN,NaN
1046866,CCC,2014J,621605,266,1,resource,NaN,NaN


In [18]:
studentVle.activity_type.unique()

array(['homepage', 'forumng', 'glossary', 'oucontent', 'quiz',
       'questionnaire', 'page', 'ouwiki', 'subpage', 'url', 'resource',
       'dualpane', 'ouelluminate', 'dataplus', 'sharedsubpage',
       'oucollaborate', 'externalquiz', 'folder', 'repeatactivity',
       'htmlactivity'], dtype=object)

In [19]:
types = ['homepage', 'forumng', 'glossary', 'oucontent', 'quiz',
       'questionnaire', 'page', 'ouwiki', 'subpage', 'url', 'resource',
       'dualpane', 'ouelluminate', 'dataplus', 'sharedsubpage',
       'oucollaborate', 'externalquiz', 'folder', 'repeatactivity',
       'htmlactivity']

def extract_features(df):

    features = {}
    
    for act_type in types:
        total = df.query(f"activity_type == '{act_type}'")
        preCourse = total.query("date < 0")

        features[f"{act_type}Clicks"] = len(total),
        features[f"{act_type}Interactions"] = total.sum_click.sum()
        features[f"{act_type}PreCourseInteractions"] = len(preCourse)
        features[f"{act_type}PreCourseInteractionsClicks"] = preCourse.sum_click.sum()
    
    return pd.Series(features)

features = studentVle.groupby(["code_module", "code_presentation", "id_student"]).apply(extract_features)
features

homepageClicks  homepageInteractions  \
code_module code_presentation id_student                                        
AAA         2013J             11391               (40,)                   138   
                              28400               (80,)                   324   
                              30268               (12,)                    59   
                              31604              (121,)                   432   
                              32885               (68,)                   204   
...                                                 ...                   ...   
GGG         2014J             2640965              (5,)                    22   
                              2645731             (35,)                   167   
                              2648187             (22,)                    63   
                              2679821             (13,)                    65   
                              2684003             (34,)                   120   

                                          homepagePreCourseInteractions  \
code_module code_presentation id_student                                  
AAA         2013J             11391                                   1   
                              28400                                   7   
                              30268                                   6   
                              31604                                   5   
                              32885                                   8   
...                                                                 ...   
GGG         2014J             2640965                                 1   
                              2645731                                 0   
                              2648187                                 1   
                              2679821                                 2   
                              2684003                                 8   

                                          homepagePreCourseInteractionsClicks  \
code_module code_presentation id_student                                        
AAA         2013J             11391                                         7   
                              28400                                        46   
                              30268                                        32   
                              31604                                        30   
                              32885                                        52   
...                                                                       ...   
GGG         2014J             2640965                                       2   
                              2645731                                       0   
                              2648187                                       6   
                              2679821                                      17   
                              2684003                                      40   

                                         forumngClicks  forumngInteractions  \
code_module code_presentation id_student                                      
AAA         2013J             11391              (52,)                  193   
                              28400             (163,)                  417   
                              30268              (30,)                  126   
                              31604             (197,)                  634   
                              32885              (77,)                  194   
...                                                ...                  ...   
GGG         2014J             2640965             (0,)                    0   
                              2645731            (14,)                   65   
                              2648187             (0,)                    0   
                              2679821            (17,)                  118   
         

In [20]:
# features.to_csv("../../data/interim/studentVleFeatures.csv")